# Assignment 2: Neural Networks and Optimization (33 marks total)
### Due: October 3 at 11:59pm

### Name: 

### In this assignment, you will need to write code that uses a linear model and a neural network to perform a regression task. You will also be asked to describe the process by which you came up with the code. More details can be found below. Please cite any websites or AI tools that you used to help you with this assignment.

## Part 1: Linear Regression vs. Neural Network

For this assignment, we will be using the concrete example from yellowbrick. We will be evaluating how well neural networks perform compared to linear regression.

### Step 0: Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import warnings
warnings.filterwarnings(action='ignore')

### Step 1: Data Input (2 marks)

The data used for this task can be imported using the yellowbrick library: 
https://www.scikit-yb.org/en/latest/api/datasets/concrete.html

Use the yellowbrick function `load_concrete()` to load the concrete dataset into the feature matrix `X` and target vector `y`.

*Note: the yellowbrick library is not included in the default Anaconda installation, so you will need to install it*

In [ ]:
# TO DO: Import concrete dataset from yellowbrick library (0.5 marks)

# TO DO: Print size and type of X and y (0.5 marks)


In [ ]:
# TO DO: Inspect the first few columns of the imported feature matrix (0.5 marks)


In [ ]:
# TO DO: Use .describe() to inspect the mean and variance of each feature (0.5 marks)


### Step 2: Data Processing (2 marks)

Check if there are any missing values and fill them in if necessary. Remove any non-numeric columns.

In [ ]:
# TO DO: Process the data - fill-in any missing values and remove any non-numeric columns (0.5 marks)


You can add the target vector and the feature matrix together and inspect the relationship between the features and target using a `pairplot()`

In [ ]:
# TO DO: Add the target vector and the feature matrix together (0.5 marks)


In [ ]:
# TO DO: Use pairplot() (0.5 marks)


The concrete data is already be split into the feature matrix and target vector. The next step is to split the data into training and testing subsets. For this assignment, you can use `train_test_split()` with `random_state=0`

In [ ]:
# TO DO: Split the data into training and testing data (0.5 marks)


Looking at the mean and variance of the dataset, it is clear that the features have a wide range of values. You can use the code below to scale the feature matrix

*Note: `StandardScaler()` scales the data to a mean of 0 and a variance of 1*

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
train_scaled = scaler.fit_transform(train_features)
test_scaled = scaler.transform(test_features)

### Step 3: Implement Machine Learning Model (2 marks)

1. Import `LinearRegression` from sklearn
2. Instantiate model `LinearRegression()`
3. Implement the machine learning model with the scaled data

In [ ]:
# TO DO: ADD YOUR CODE HERE


### Step 4: Validate Model (3 marks)

Calculate the training and validation accuracy using mean squared error.

In [ ]:
# TO DO: ADD YOUR CODE HERE (2 marks)


In [ ]:
# TO DO: Print the results (1 mark)


## Part B: Neural Network

Now we will repeat the above analysis using a neural network. For this assignment, we will be using the PyTorch library.

In [ ]:
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader

### Step 2: Data Processing (2 marks)

To make this analysis easier, we can convert the data into tensors.

In [ ]:
# TO DO: Convert training and testing data to tensors (1 mark)


In [ ]:
# TO DO: Print the size of the training features and labels (1 mark)


The labels must be changed from a vector to a 2-D array to make sure that the math works properly. Use the provided code below to fix this issue:

In [ ]:
y_train = y_train.unsqueeze(1)
y_test = y_test.unsqueeze(1)
print(y_train.size())

### Step 3: Implement Neural Network (10 marks)

For this assignment, we will use the SGD optimizer with the following parameters:
- Initial learning rate = 0.001
- Momentum = 0.9

We will use the same learning rate schedule that was used in the Backpropagation Example on D2L.

For the optimization process, we will need to both train and test the data for each epoch. It is up to you how you want to organize your code for this process. You will need to include the following steps:
- Load your data using the `DataLoader()`
- Define a model with two hidden layers with 50 hidden units each
- Choose your loss function
- initialize your optimizer
- Set up your training and testing loops for 100 epochs
    - Remember that you must go through the training and testing code for each epoch sequentially
- Print out the average training and testing loss for each epoch (or plot in graph)

In [ ]:
from torch.optim.lr_scheduler import StepLR
# TO DO: ADD YOUR CODE HERE


### Questions (6 marks)
1. How did the results from the linear model compare to the results from the neural network? Why do you think one performed better than the other?
1. If you run the optimization/backpropagation code multiple times, you will see that you get different loss values. Why is this?
1. Compare the results from SGD to using Adam with default parameters and a constant learning rate of 0.01. Which model would you select to use and why?

*ANSWER HERE*


### Process Description (4 marks)
Please describe the process you used to create your code. Cite any websites or generative AI tools used. You can use the following questions as guidance:
1. Where did you source your code?
1. In what order did you complete the steps?
1. If you used generative AI, what prompts did you use? Did you need to modify the code at all? Why or why not?
1. Did you have any challenges? If yes, what were they? If not, what helped you to be successful?

*DESCRIBE YOUR PROCESS HERE - BE SPECIFIC*

## Part 2: Reflection (2 marks)
Include a sentence or two about:
- what you liked or disliked,
- found interesting, confusing, challenging, motivating
while working on this assignment.


*ADD YOUR THOUGHTS HERE*